In [3]:
import warnings
warnings.filterwarnings('ignore')
import keras
import pandas as pd
from sklearn.model_selection import train_test_split
import re, string, unicodedata
import nltk
#nltk.download('popular')
from nltk.stem.snowball import SnowballStemmer
#import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import numpy as np
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
from numpy import mean, std
from sklearn.metrics import classification_report
import plotly.graph_objs as go
import plotly.plotly as py
#import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
import matplotlib.pyplot as plt




Using TensorFlow backend.


In [4]:
data = pd.read_csv('google-maps-single-dataset-for-conference.csv',encoding='ISO-8859-1')
data= data[pd.notnull(data['Claim_Type'])]
print(len(data))

data = data.replace("neu ", "neu")
data = data.replace("supporting ", "supporting")
data = data.replace("attacking ", "attacking")
data = data.replace("atacking ", "attacking")
data = data.replace("attacknig ", "attacking")

data= data[data.Rationale_Type=='claim']
print(data.Claim_Type.value_counts())
#data = data.replace("issue ", "issue")
data.head()

2077
neu           772
supporting    762
attacking     542
Name: Claim_Type, dtype: int64


,comment_Text,comment_author,comment_id,comment_upvotes,parent_id,Rationale_Type,Claim_Type,Have_Rationale
1,People still do racial checkbox ads?,PubicLouseInDaHouse,dfadazi,-1,60v5za,claim,neu,no
2,What?,Isvara,dfade5q,1,dfadazi,claim,neu,no
3,90s power rangers.,PubicLouseInDaHouse,dfadlgi,1,dfade5q,claim,neu,no
5,https://support.google.com/plus/answer/2998354...,sparr,dfa365t,-1,60v5za,claim,supporting,yes
6,This is really cool for people who have friends.,Unidan_nadinU,df9vhad,-1,60v5za,claim,supporting,yes


In [5]:
def print_plot(index):
    example = data[data.index == index][['comment_Text', 'Rationale_Type']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Rationale_Type:', example[1])


In [6]:
print_plot(60)

> gender equality in any and all professions.

Hahah uh... what?

EDIT: Also I wasn't the one to downvote you above. No clue what that's about.
Rationale_Type: claim


In [7]:
#df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\<\>\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z =#+_]')
#STOPWORDS = set(stopwords.words('english'))

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_url(text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    return text

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = strip_html(text) 
    text = remove_url(text)
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    #text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    #text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
data['comment_Text'] = data['comment_Text'].apply(clean_text)
#df['Consumer complaint narrative'] = df['Consumer complaint narrative'].str.replace('\d+', '')

In [8]:
print_plot(60)

  gender equality in any and all professionshahah uh whatedit also i wasnt the one to downvote you above no clue what thats about
Rationale_Type: claim


In [52]:
#print(data.Rationale_Type.value_counts())
X= data.comment_Text.values.astype('U')
y=data.Claim_Type.values.astype('U')
rationale_type=data.Rationale_Type
rationale_type_name= data.Claim_Type.unique()
print(rationale_type_name)

['neu' 'supporting' 'attacking']


In [62]:
vocab_size= 10000
MAX_SEQUENCE_LENGTH = 1000
batch_size= 12
epochs= 10
test_accuracy =[]
report_accuracy= []

def evaluate_ANN_Model(trainX,trainy, testX, testy):
    tokenizer= Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(trainX)
    X_train_matrix_tfidf= tokenizer.texts_to_matrix(trainX,mode='tfidf')
    X_test_matrix_tfidf= tokenizer.texts_to_matrix(testX, mode='tfidf')

    encoder= LabelBinarizer()
    encoder.fit(trainy)
    Y_train_matrix_tfidf= encoder.transform(trainy)
    Y_test_matrix_tfidf= encoder.transform(testy)
    smote = SMOTE()
    X_sm_tfidf, y_sm_tfidf = smote.fit_sample(X_train_matrix_tfidf, Y_train_matrix_tfidf)
    
    #print(vocab_size)
    model = Sequential()
    #input layer
    model.add(Dense(16, input_shape=(vocab_size, )))
    model.add(Activation('relu'))
    model.add(Dropout(0.6))
    #hidden layer
    #odel.add(Dense(32))
    #odel.add(Activation('relu'))
    #odel.add(Dropout(0.3))
    #output layer
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
    #model. summary()
    #print(model.metrics_names)
    #fit model 
    model.fit(X_sm_tfidf,y_sm_tfidf, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2)
    #evaluate model
    score= model.evaluate(X_test_matrix_tfidf, Y_test_matrix_tfidf, batch_size=batch_size, verbose=1)
    test_accuracy= score[1]
    #predict model accuracy on testing data
    pred = model.predict(X_test_matrix_tfidf, batch_size=12, verbose=1)
    predicted = np.argmax(pred, axis=1)
    report_accuracy = precision_recall_fscore_support(np.argmax(Y_test_matrix_tfidf, axis=1), predicted)
    return model, test_accuracy, report_accuracy



    

In [63]:
#Kfold ANN with TFIDF

kf= KFold(n_splits=3)
#curr_fold= 0
alg_accurcy_ANN_tfidf =[]
alg_testing_accurcy_ANN_tfidf =[]

for train_idx, test_idx in kf.split(X):
    X_train, X_test= X[train_idx], X[test_idx]
    y_train, y_test= y[train_idx], y[test_idx]
        
    model, test_accuracy,report_accuracy = evaluate_ANN_Model(X_train,y_train, X_test, y_test )
    print ("the training accuracy is",test_accuracy )
    alg_accurcy_ANN_tfidf.append(test_accuracy)
    print ("the testing accuracy is",report_accuracy )
    alg_testing_accurcy_ANN_tfidf.append(report_accuracy)
    

average_accuracy_ANN_training= np.mean(alg_accurcy_ANN_tfidf, axis=0)
print("the average Training Accuracy of ANN-TFIDF is",average_accuracy_ANN_training) 

average_accuracy_ANN_testing= np.mean(alg_testing_accurcy_ANN_tfidf, axis=0)
print("the averag Validation accuracy of ANN-TFIDF is",average_accuracy_ANN_testing)



Train on 1341 samples, validate on 336 samples
Epoch 1/10
1341/1341 [==============================] - 17s 12ms/step - loss: 1.1167 - acc: 0.3594 - val_loss: 1.0392 - val_acc: 0.4464
Epoch 2/10
1341/1341 [==============================] - 1s 1ms/step - loss: 0.9846 - acc: 0.4884 - val_loss: 0.9592 - val_acc: 0.5536
Epoch 3/10
1341/1341 [==============================] - 1s 800us/step - loss: 0.8792 - acc: 0.5839 - val_loss: 0.8727 - val_acc: 0.6458
Epoch 4/10
1341/1341 [==============================] - 1s 668us/step - loss: 0.7733 - acc: 0.6339 - val_loss: 0.7852 - val_acc: 0.6964
Epoch 5/10
1341/1341 [==============================] - 1s 625us/step - loss: 0.6760 - acc: 0.6980 - val_loss: 0.6838 - val_acc: 0.7649
Epoch 6/10
1341/1341 [==============================] - 1s 645us/step - loss: 0.5708 - acc: 0.7547 - val_loss: 0.5849 - val_acc: 0.8155
Epoch 7/10
1341/1341 [==============================] - 1s 593us/step - loss: 0.5001 - acc: 0.7934 - val_loss: 0.5296 - val_acc: 0.8423
Epo

In [56]:
vocab_size= 10000
MAX_SEQUENCE_LENGTH = 1000
batch_size= 12
epochs= 10
test_accuracy =[]
report_accuracy= []

def evaluate_ANN_Model(trainX,trainy, testX, testy):
    
    
    tokenizer= Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(trainX)
    X_train_matrix_binary= tokenizer.texts_to_matrix(trainX,mode='binary')
    X_test_matrix_binary= tokenizer.texts_to_matrix(testX, mode='binary')

    encoder= LabelBinarizer()
    encoder.fit(trainy)
    Y_train_matrix_binary= encoder.transform(trainy)
    Y_test_matrix_binary= encoder.transform(testy)
    
    
    smote = SMOTE()
    X_sm_binary, y_sm_binary = smote.fit_sample(X_train_matrix_binary, Y_train_matrix_binary)
    
    MAX_SEQUENCE_LENGTH = 1000
    #print(vocab_size)
    model = Sequential()
    model.add(Dense(16, input_shape=(vocab_size, )))
    model.add(Activation('relu'))
    model.add(Dropout(0.6))
    #hidden layer
    #model.add(Dense(16))
    #model.add(Activation('relu'))
    #model.add(Dropout(0.5))

    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
    #model. summary()
    #print(model.metrics_names)

    model.fit(X_sm_binary,y_sm_binary, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2)
    score= model.evaluate(X_test_matrix_binary, Y_test_matrix_binary, batch_size=batch_size, verbose=1)
    test_accuracy= score[1]
    #predict model accuracy on testing data
    pred = model.predict(X_test_matrix_binary, batch_size=12, verbose=1)
    predicted = np.argmax(pred, axis=1)
    report_accuracy = precision_recall_fscore_support(np.argmax(Y_test_matrix_binary, axis=1), predicted)
    return model, test_accuracy, report_accuracy  

    

In [57]:
#Kfold ANN with mode=binary

kf= KFold(n_splits=2)
#curr_fold= 0
alg_accurcy_ANN_binary =[]
alg_testing_accurcy_ANN_binary =[]

for train_idx, test_idx in kf.split(X):
    X_train, X_test= X[train_idx], X[test_idx]
    y_train, y_test= y[train_idx], y[test_idx]
        
    model, test_accuracy,report_accuracy = evaluate_ANN_Model(X_train,y_train, X_test, y_test )
    print ("the training accuracy is",test_accuracy )
    alg_accurcy_ANN_binary.append(test_accuracy)
    print ("the testing accuracy is",report_accuracy )
    alg_testing_accurcy_ANN_binary.append(report_accuracy)
    

average_accuracy_ANN_binary= np.mean(alg_accurcy_ANN_tfidf, axis=0)
print("the average Training Accuracy of ANN-Binary is",average_accuracy_ANN_binary) 

average_accuracy_ANN_binary= np.mean(alg_testing_accurcy_ANN_tfidf, axis=0)
print("the averag Validation accuracy of ANN-Binary is",average_accuracy_ANN_binary)





Train on 1010 samples, validate on 253 samples
Epoch 1/10
1010/1010 [==============================] - 5s 5ms/step - loss: 1.0960 - acc: 0.4069 - val_loss: 1.0753 - val_acc: 0.4704: 1s - loss: 1.1003 - acc
Epoch 2/10
1010/1010 [==============================] - 2s 2ms/step - loss: 1.0294 - acc: 0.5287 - val_loss: 1.0584 - val_acc: 0.4032
Epoch 3/10
1010/1010 [==============================] - 2s 2ms/step - loss: 0.9470 - acc: 0.5980 - val_loss: 0.9904 - val_acc: 0.5573
Epoch 4/10
1010/1010 [==============================] - 1s 1ms/step - loss: 0.8424 - acc: 0.6713 - val_loss: 0.9372 - val_acc: 0.5771
Epoch 5/10
1010/1010 [==============================] - 1s 787us/step - loss: 0.7418 - acc: 0.7198 - val_loss: 0.8872 - val_acc: 0.6206
Epoch 6/10
1010/1010 [==============================] - 1s 726us/step - loss: 0.6688 - acc: 0.7505 - val_loss: 0.7919 - val_acc: 0.6996
Epoch 7/10
1010/1010 [==============================] - 1s 631us/step - loss: 0.5938 - acc: 0.7950 - val_loss: 0.7407 - 

In [60]:
vocab_size= 10000
MAX_SEQUENCE_LENGTH = 1000
batch_size= 12
epochs= 10
test_accuracy =[]
report_accuracy= []

def evaluate_ANN_Model(trainX,trainy, testX, testy):
    
    tokenizer= Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(trainX)
    X_train_matrix_count= tokenizer.texts_to_matrix(trainX,mode='count')
    X_test_matrix_count= tokenizer.texts_to_matrix(testX, mode='count')

    encoder= LabelBinarizer()
    encoder.fit(trainy)
    Y_train_matrix_count= encoder.transform(trainy)
    Y_test_matrix_count= encoder.transform(testy)     
    
    smote = SMOTE('minority')
    X_sm_count, y_sm_count = smote.fit_sample(X_train_matrix_count, Y_train_matrix_count)
    
    model = Sequential()
    model.add(Dense(16, input_shape=(vocab_size, )))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
    #model. summary()
    #print(model.metrics_names)
    from sklearn.utils import class_weight
    class_weight= class_weight.compute_class_weight('balanced', np.unique(rationale_type),rationale_type)
    #print(class_weight)
    model.fit(X_sm_count,y_sm_count, batch_size=batch_size, class_weight=class_weight, epochs=epochs, verbose=1, validation_split=0.3)
    score=model.evaluate(X_test_matrix_count, Y_test_matrix_count, batch_size=batch_size, verbose=1)
    test_accuracy= score[1]
    #predict model accuracy on testing data
    pred = model.predict(X_test_matrix_count, batch_size=12, verbose=1)
    predicted = np.argmax(pred, axis=1)
    report_accuracy = precision_recall_fscore_support(np.argmax(Y_test_matrix_count, axis=1), predicted)
    return model, test_accuracy, report_accuracy  

    

In [61]:
#Kfold ANN with mode=count

kf= KFold(n_splits=2)
#curr_fold= 0
alg_accurcy_ANN_count =[]
alg_testing_accurcy_ANN_count =[]

for train_idx, test_idx in kf.split(X):
    X_train, X_test= X[train_idx], X[test_idx]
    y_train, y_test= y[train_idx], y[test_idx]
        
    model, test_accuracy,report_accuracy = evaluate_ANN_Model(X_train,y_train, X_test, y_test )
    print ("the training accuracy is",test_accuracy )
    alg_accurcy_ANN_count.append(test_accuracy)
    print ("the testing accuracy is",report_accuracy )
    alg_testing_accurcy_ANN_count.append(report_accuracy)
    

average_accuracy_ANN_count= np.mean(alg_accurcy_ANN_count, axis=0)
print("the average Training Accuracy of ANN-Binary is",average_accuracy_ANN_count) 

average_accuracy_ANN_count= np.mean(alg_testing_accurcy_ANN_count, axis=0)
print("the averag Validation accuracy of ANN-Binary is",average_accuracy_ANN_count)


Train on 829 samples, validate on 356 samples
Epoch 1/10
829/829 [==============================] - 8s 10ms/step - loss: 1.0949 - acc: 0.3776 - val_loss: 1.0875 - val_acc: 0.3933
Epoch 2/10
829/829 [==============================] - 1s 2ms/step - loss: 1.0282 - acc: 0.5476 - val_loss: 1.0540 - val_acc: 0.4354
Epoch 3/10
829/829 [==============================] - 2s 2ms/step - loss: 0.9031 - acc: 0.6779 - val_loss: 1.0102 - val_acc: 0.4944
Epoch 4/10
829/829 [==============================] - 1s 1ms/step - loss: 0.7941 - acc: 0.7358 - val_loss: 1.0095 - val_acc: 0.4860
Epoch 5/10
829/829 [==============================] - 1s 892us/step - loss: 0.6576 - acc: 0.7973 - val_loss: 0.9792 - val_acc: 0.5421
Epoch 6/10
829/829 [==============================] - 1s 774us/step - loss: 0.5833 - acc: 0.8287 - val_loss: 0.9451 - val_acc: 0.5674
Epoch 7/10
829/829 [==============================] - 1s 719us/step - loss: 0.4724 - acc: 0.8842 - val_loss: 0.9382 - val_acc: 0.5787
Epoch 8/10
829/829 [===